What we learn at the Multimodal Learning Analytics Workshop
==============================

## Plan Multimodal Learning Analytics Studies ##

<img src="MDC.png">

# Capture and Process Different Modalities #

## Images ##

In [ ]:
import time 
import requests
import cv2
import operator
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import clear_output
import speech_recognition as sr
import time
import xml.etree.ElementTree as etree 
from pylab import *
from sklearn import tree
import pandas as pd
from IPython.display import Image  
from sklearn.externals.six import StringIO 
import pydotplus

# Define the API to use
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = 'c0958b1e63134377960c2796e82aa14e'
_maxNumRetries = 10
# Functions to use the API
def processRequest( json, data, headers ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = None )

        if response.status_code == 429: 

            print("Message: %s" % ( response.json()['error']['message'] ))

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print('Error: failed after retrying!')
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print("Error code: %d" % ( response.status_code ))
            print("Message: %s" % ( response.json()['error']['message'] ))

        break
        
    return result

# Fuction to add the emotion to the image
def renderResultOnImage( result, img ):
    
    """Display the obtained results onto the input image"""
    
    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        cv2.rectangle( img,(faceRectangle['left'],faceRectangle['top']),
                           (faceRectangle['left']+faceRectangle['width'], faceRectangle['top'] + faceRectangle['height']),
                       color = (255,0,0), thickness = 5 )


    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        currEmotion = max(currFace['scores'].items(), key=operator.itemgetter(1))[0]


        textToWrite = "%s" % ( currEmotion )
        cv2.putText( img, textToWrite, (faceRectangle['left'],faceRectangle['top']-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1 )

In [ ]:
# Read a static image and assign emotion
pathToFileInDisk = r'MLA.png'
with open( pathToFileInDisk, 'rb' ) as f:
    data = f.read()

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None

result = processRequest( json, data, headers )

for currFace in result:
        faceRectangle = currFace['faceRectangle']
        currEmotion = max(currFace['scores'].items(), key=operator.itemgetter(1))[0]
        print(currEmotion)



# Load the original image from disk
data8uint = np.fromstring( data, np.uint8 ) # Convert string to an unsigned int array
img = cv2.cvtColor( cv2.imdecode( data8uint, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )

renderResultOnImage( result, img )

ig, ax = plt.subplots(figsize=(15, 20))
ax.imshow( img )

## Video ##

In [ ]:
#We obtain the object to be tracked (calculator)
MIN_MATCH_COUNT = 2

cap = cv2.VideoCapture('MLA2.mp4')
img1 = cv2.imread("MLA2.png",0)  # queryImage
img1show = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)

frame = 0

#For each frame
while(frame<200):
    cap.set(cv2.CAP_PROP_POS_FRAMES,frame)
    ret, img2 = cap.read()
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    x = -1
    y = -1
    degreeCorrection = 20
    student = -1

    frame = frame + 25

    #Create a SIFT feature extractor
    try:
        sift = cv2.xfeatures2d.SIFT_create()
        kp1, des1 = sift.detectAndCompute(img1,None)
        kp2, des2 = sift.detectAndCompute(img2,None)

        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)

        matches = flann.knnMatch(des1,des2,k=2)

        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)


        if len(good)>=MIN_MATCH_COUNT:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

            TM=cv2.estimateRigidTransform(src_pts, dst_pts,False)
            newrow = [0,0,1]
            TM = np.vstack([TM, newrow])

            h,w = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,TM)

            cv2.polylines(img2,[np.int32(dst)],True,(255,255,255),5)
            degree = math.degrees(math.acos(TM[0][0]))
            degree=degree+degreeCorrection

            centerx = (dst[0][0][0]+dst[2][0][0])/2
            centery = (dst[0][0][1]+dst[2][0][1])/2

            x = centerx
            y = centery
            cv2.circle(img2, (np.int32(centerx),np.int32(centery)), 10, (255,255,255),10) 

            #student = 0
            height,width = img2.shape[0],img2.shape[1]
            n = (centery) - (math.tan(math.pi/2-math.acos(TM[0][0])) * (width-centerx))
            x0=(-n)/math.tan(math.radians(90-degree))

            xf = x
            yf = y/2

            if x0>-1 and x0<width+1:
                if x0<180:
                    student = 1
                if x0>179 and x0<701:
                    student = 2
                if x0>700:
                    student = 3
            else:
                if x0<0:
                    student = 1
                if x0>width:
                    student = 3

            cv2.line(img2, (np.int32(centerx),np.int32(centery)), (np.int32(width-x0),0), (237,19,19),5)

        else:
            print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    except:
        pass 

   
    fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    img2show = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    plt.imshow(img2show)
    plt.show()
    clear_output(wait=True)
    

cap.release()

## Audio ##

In [ ]:
from os import path
from os import listdir
directory="afiles"

text=""
r = sr.Recognizer()
files=listdir(directory)


for file in files:
    if not "audio" in file:
         files.remove(file)
for i in range(1,len(files)+1):
    print(i)
    AUDIO_FILE = directory+"/"+"audiofile"+str(i)+".wav"
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source) # read the entire audio file
        # use the audio file as the audio source
    # recognize speech using Microsoft Bing Voice Recognition
    BING_KEY = "8638a1837c0946f799ecd77e011d97d9" # Microsoft Bing Voice Recognition API keys 32-character lowercase hexadecimal strings
    try:
        text = r.recognize_bing(audio, key=BING_KEY)
        print("Audio recognized")
        print("Recognized text:", text)
        
    except sr.UnknownValueError:
        print("Audio not understood")
    except sr.RequestError as e:
        print("Error with service; {0}".format(e))
    time.sleep(10)



# Digital Pens #

In [ ]:
directory="digipen/"
student="S1"
treeSessions = etree.parse(directory+student+"/"+"data_session.xml")
rootSessions=treeSessions.getroot()
notes = rootSessions[0]
traces =[]
pages = []
for note in notes:
    page=[]
    startime = note.get('starttime')
    endtime = note.get('endtime')
    notefile = note[0].text
    treeNote = etree.parse(directory+student+"/"+notefile)
    rootNote = treeNote.getroot()
    noteinfo = rootNote[0]
    for trace in noteinfo:
        tracedata=[]
        for point in trace:
            pointdata={}
            pointdata["x"] = float(point[0].text)
            pointdata["y"] = float(point[1].text)
            pointdata["timestamp"] = long(point[2].text)
            pointdata["force"] = float(point[3].text)
            tracedata.append(pointdata)
        traces.append(tracedata)
        page.append(tracedata)
    pages.append(page)

for page in pages:
    figure(figsize=(10*1.5,7*1.5))
    for trace in page:
        x =[]
        y=[]
        for point in trace:
            x.append(int(point.get("x")))
            y.append(int(point.get("y")))
        plot(y,x)
    show()

## Fusion and Decisions ##

In [ ]:
data=pd.read_csv("DataExpert.csv",sep=",")

data

In [ ]:
features=['PCU', 'DHT', 'SDDHT', 'TM', 'NOI', 'TSD', 'TNM', 'PNM', 'TMTM', 'PMTM', 'TCP', 'TNS', 'NOL', 'NOR', 
               'NOC', 'NOE', 'NOA', 'ASPL', 'ANP', 'ASD', 'ASTL', 'ASP', 'SS', 'MN', 'MM', 'MC', 'MMO', 
               'LMO', 'LMV', 'MTM', 'MI']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(data[features], data['Expert'])
dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data, feature_names=features)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png()) 